# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
# print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# FOR THIS QUERY WE AIMED TO CREATE A NEW KEYSPACVE IF NOT EXISTS
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };
        """
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## for this query sessionid and iteminsession are considered as the composite particion key. we these two columns
## to identify where data will reside. we have to ask for these two kwy to guaranty we grouped data in the right way
query = """
    CREATE TABLE IF NOT EXISTS artists_by_sessionId (
        sessionId INT,
        itemInSession INT,
        artist TEXT,
        song TEXT,
        length FLOAT,
        PRIMARY KEY (sessionId, itemInSession)
    );
"""
try:
    session.execute(query)
except Exception as e:
    print(e)   

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artists_by_sessionId (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## this query response to requirements, we make sure we ask for columns that compound the compound key
query = """
    SELECT
        artist,
        song,
        length
    FROM
        artists_by_sessionId
    WHERE 
        sessionId = 637 AND itemInSession = 4"""
try:
    rows = session.execute(query)
    for row in rows:
        print (row.artist, row.song, row.length)
except Exception as e:
    print(e)

Billy Talent Voices Of Violence (Album Version) 190.37994384765625


In [11]:
## this query we use userid and sessionid as composite Partition Key also we add itemInSession, firstName, lastName to sort the data, also 
## we can incluse those columns in the query. Nota: must be in order
query = """
    CREATE TABLE IF NOT EXISTS artists_by_userid (
        userId INT,
        sessionId INT,
        itemInSession INT,
        firstName TEXT,
        lastName TEXT,
        artist TEXT,
        song TEXT,
        PRIMARY KEY ((userid, sessionid), itemInSession, firstName, lastName)
    );

"""
try:
    session.execute(query)
except Exception as e:
    print(e) 
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artists_by_userid (userId, sessionId, itemInSession, firstName, lastName, artist, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[1], line[2], line[0], line[9]))

In [13]:
## We responds to the the second requeriment, we ask just for two columns
query = """
    SELECT
        artist,
        song,
        firstName,
        lastName
    FROM
        artists_by_userid
    WHERE 
        userid = 10 and sessionid = 182"""
try:
    rows = session.execute(query)
    for row in rows:
        print (row.artist, row.song, row.firstname, row.lastname)
except Exception as e:
    print(e)

Down To The Bone Keep On Keepin' On Sylvie F
Three Drives Greece 2000 Sylvie F
Sebastien Tellier Kilometer Sylvie F
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie F


In [14]:
## For this query we use song and userId as composite Partition Key, becuase we want to ask for songs by user id 
query = """
    CREATE TABLE IF NOT EXISTS artists_by_song (
        song TEXT,
        userId INT,
        firstName TEXT,
        lastName TEXT,
        PRIMARY KEY (song, userId)
    );
"""
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artists_by_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[2]))

In [16]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## query provide name of the artist 
query = """
    SELECT
        firstName,
        lastName
    FROM
        artists_by_song
    WHERE 
        song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query)
    for row in rows:
        print (row.firstname, row.lastname)
except Exception as e:
    print(e)

Jacqueline F
Tegan F
Sara F


### Drop the tables before closing out the sessions

In [17]:
## Drop the tables before closing out the sessions
query = "DROP TABLE artists_by_sessionId"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE artists_by_userid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE artists_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()